In [1]:
import sys
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

import pandas as pd
import numpy as np

In [2]:
TIMEOUT = 5

driver = None

In [6]:
df = pd.read_csv('test.csv', encoding='utf-8-sig')
search_index = df.index[df['p_review'] == '리뷰 없음'].tolist()
# df[df['p_review'] == '리뷰 없음']['p_name']
df[df['p_review'] == '리뷰 없음']

,Unnamed: 0,p_name,p_address,p_rating,p_category,p_tag,p_review,p_url
2,2,한옥 수연이네,울산 중구 새즈믄해거리 47,5.0,게스트하우스,"['#가족펜션', '#독채펜션', '#물놀이', '#바베큐장', '#불멍', '#온...",리뷰 없음,https://place.map.kakao.com/431795702
3,3,게스트하우스유,울산 남구 대학로15번길 6 문수빌라 102호,0.0,게스트하우스,태그 없음,리뷰 없음,https://place.map.kakao.com/19889650
5,5,심청골,울산 북구 양정동,4.0,계곡,태그 없음,리뷰 없음,https://place.map.kakao.com/11110570
6,6,천전리각석계곡,울산 울주군 두동면 천전리,0.0,계곡,태그 없음,리뷰 없음,https://place.map.kakao.com/12639217
7,7,배내골,울산 울주군 상북면 이천리,0.0,계곡,['#물놀이'],리뷰 없음,https://place.map.kakao.com/25247099
...,...,...,...,...,...,...,...,...
1188,1188,옥류천이야기길,NaN,0.0,NaN,태그 없음,리뷰 없음,https://place.map.kakao.com/26563873
1189,1189,강동사랑길 2코스,NaN,4.0,NaN,태그 없음,리뷰 없음,https://place.map.kakao.com/26564336
1191,1191,입화산참살이숲 누리길 2코스,NaN,0.0,NaN,태그 없음,리뷰 없음,https://place.map.kakao.com/26564078
1194,1194,태화강100리길 2코스,NaN,0.0,NaN,태그 없음,리뷰 없음,https://place.map.kakao.com/26558730


In [37]:
info_list = driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div:nth-child(2) > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd').find_elements(By.CSS_SELECTOR, 'div.y7PRA')
for info in info_list:
  print(df.loc[search_index[4], 'p_name'], info.find_element(By.CSS_SELECTOR, 'div.qBF1Pd.fontHeadlineSmall > span').text)
  if df.loc[search_index[4], 'p_name'] in info.find_element(By.CSS_SELECTOR, 'div.qBF1Pd.fontHeadlineSmall > span').text:
    # info.find_element(By.CSS_SELECTOR, 'div.qBF1Pd.fontHeadlineSmall > span').click()
    info.find_element(By.CSS_SELECTOR, 'div.qBF1Pd.fontHeadlineSmall > span').send_keys(Keys.ENTER)
    # driver.execute_script('arguments[0].click();', info.find_element(By.CSS_SELECTOR, 'div.qBF1Pd.fontHeadlineSmall > span'))
    enter = True
    break
  else:
    continue

배내골 워크샵펜션
배내골 배내골


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=110.0.5481.180)
Stacktrace:
Backtrace:
	(No symbol) [0x00F637D3]
	(No symbol) [0x00EF8B81]
	(No symbol) [0x00DFB230]
	(No symbol) [0x00E28586]
	(No symbol) [0x00E276A7]
	(No symbol) [0x00E4B41C]
	(No symbol) [0x00E22B96]
	(No symbol) [0x00E4B774]
	(No symbol) [0x00E61215]
	(No symbol) [0x00E4B216]
	(No symbol) [0x00E20D97]
	(No symbol) [0x00E2253D]
	GetHandleVerifier [0x011DABF2+2510930]
	GetHandleVerifier [0x01208EC1+2700065]
	GetHandleVerifier [0x0120C86C+2714828]
	GetHandleVerifier [0x01013480+645344]
	(No symbol) [0x00F00FD2]
	(No symbol) [0x00F06C68]
	(No symbol) [0x00F06D4B]
	(No symbol) [0x00F10D6B]
	BaseThreadInitThunk [0x768F7D69+25]
	RtlInitializeExceptionChain [0x77D8BB9B+107]
	RtlClearBits [0x77D8BB1F+191]


In [ ]:
def search_review(driver, df, index, p_name):
  place = []
  user = []
  rating = []
  comment = []
  enter = False
  try:
    info_list = driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div:nth-child(2) > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd').find_elements(By.CSS_SELECTOR, 'div.bfdHYd.Ppzolf.OFBs3e')
    for info in info_list:
      if df.loc[index, 'p_name'] in info.find_element(By.CSS_SELECTOR, 'div.qBF1Pd.fontHeadlineSmall > span').text:
        info.click()
        enter = True
        break
      else:
        continue
    if enter == True:
      try:
        driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span[2]/span[1]/span').click()
        elem = driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf')
        for i in range(20):
          driver.execute_script("arguments[0].scrollBy(0, 2400)", elem)
          sleep(0.5)
      
        review_boxes = driver.find_elements(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div:nth-child(8) > div')
      
        review_count = driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div.PPCwl > div > div.jANrlb > div.fontBodySmall').text.replace('리뷰 ', '').replace('개', '')
        if int(review_count)<200:
          for i in range(0, (int(review_count))*3, 3):
            place.append(p_name)
            
            user_name = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.WNxzHc.qLhwHc > a > div.d4r55 > span').text
            user.append(user_name)
          
            # place_rating = int(review_boxes[i].find_element(By.CSS_SELECTOR, 'span.kvMYJc').get_attribute('aria-label').replace('별표 ', '').replace('개', ''))
            place_rating = int(review_boxes[i].find_element(By.CSS_SELECTOR, 'span.fzvQIb').text.replace('/5', ''))
            rating.append(place_rating)
          
            # place_comment = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.GHT2ce > span.wiI7pd').text
            place_comment = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.MyEned > span.wiI7pd').text
            comment.append(place_comment)
            
          review_df = pd.DataFrame({
              'place_id':place,
              'user_id':user,
              'score':rating,
              'comment':comment
          },)

          review_df.to_csv(f'./kakao_reviews/google_review/{p_name}.csv', index = False, header=True, encoding='utf-8-sig')
          return f'./kakao_reviews/google_review/{p_name}.csv'
          
        else:
          for i in range(0, 200*3, 3):
            place.append(p_name)
            
            user_name = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.WNxzHc.qLhwHc > a > div.d4r55 > span').text
            user.append(user_name)
          
            # place_rating = int(review_boxes[i].find_element(By.CSS_SELECTOR, 'span.kvMYJc').get_attribute('aria-label').replace('별표 ', '').replace('개', ''))
            place_rating = int(review_boxes[i].find_element(By.CSS_SELECTOR, 'span.fzvQIb').text.replace('/5', ''))
            rating.append(place_rating)
          
            # place_comment = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.GHT2ce > span.wiI7pd').text
            place_comment = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.MyEned > span.wiI7pd').text
            comment.append(place_comment)
            
          review_df = pd.DataFrame({
              'place_id':place,
              'user_id':user,
              'score':rating,
              'comment':comment
          },)

          review_df.to_csv(f'./kakao_reviews/google_review/{p_name}.csv', index = False, header=True, encoding='utf-8-sig')
          return f'./kakao_reviews/google_review/{p_name}.csv'
          
      except Exception as e:
        return '리뷰 없음'

  except Exception as e:
    try:
      driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span[2]/span[1]/span').click()
      elem = driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf')
      for i in range(20):
        driver.execute_script("arguments[0].scrollBy(0, 2400)", elem)
        sleep(0.5)
      
      review_boxes = driver.find_elements(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div:nth-child(8) > div')
      
      review_count = driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div.PPCwl > div > div.jANrlb > div.fontBodySmall').text.replace('리뷰 ', '').replace('개', '')
      if int(review_count)<200:
        for i in range(0, (int(review_count))*3, 3):
          place.append(p_name)
            
          user_name = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.WNxzHc.qLhwHc > a > div.d4r55 > span').text
          user.append(user_name)
          
          # place_rating = int(review_boxes[i].find_element(By.CSS_SELECTOR, 'span.kvMYJc').get_attribute('aria-label').replace('별표 ', '').replace('개', ''))
          place_rating = int(review_boxes[i].find_element(By.CSS_SELECTOR, 'span.fzvQIb').text.replace('/5', ''))
          rating.append(place_rating)
          
          # place_comment = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.GHT2ce > span.wiI7pd').text
          place_comment = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.MyEned > span.wiI7pd').text
          comment.append(place_comment)
            
        review_df = pd.DataFrame({
            'place_id':place,
            'user_id':user,
            'score':rating,
            'comment':comment
        },)

        review_df.to_csv(f'./kakao_reviews/google_review/{p_name}.csv', index = False, header=True, encoding='utf-8-sig')
        return f'./kakao_reviews/google_review/{p_name}.csv'
          
      else:
        for i in range(0, 200*3, 3):
          place.append(p_name)
            
          user_name = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.WNxzHc.qLhwHc > a > div.d4r55 > span').text
          user.append(user_name)
          
          # place_rating = int(review_boxes[i].find_element(By.CSS_SELECTOR, 'span.kvMYJc').get_attribute('aria-label').replace('별표 ', '').replace('개', ''))
          place_rating = int(review_boxes[i].find_element(By.CSS_SELECTOR, 'span.fzvQIb').text.replace('/5', ''))
          rating.append(place_rating)
          
          # place_comment = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.GHT2ce > span.wiI7pd').text
          place_comment = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.MyEned > span.wiI7pd').text
          comment.append(place_comment)
            
        review_df = pd.DataFrame({
            'place_id':place,
            'user_id':user,
            'score':rating,
            'comment':comment
        },)

        review_df.to_csv(f'./kakao_reviews/google_review/{p_name}.csv', index = False, header=True, encoding='utf-8-sig')
        return f'./kakao_reviews/google_review/{p_name}.csv'
          
    except Exception as e:
      return '리뷰 없음'

In [3]:
driver = webdriver.Chrome("./chromedriver")

driver.get("https://www.google.co.kr/maps")
driver.implicitly_wait(TIMEOUT)

C:\Users\82107\AppData\Local\Temp\ipykernel_45136\3480925466.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver")


In [ ]:
for search in search_index:
  elem = driver.find_element(By.ID, "searchboxinput")
  # elem.send_keys(df.loc[search_index[i], 'p_category']+' '+df.loc[search_index[i], 'p_name'])
  elem.send_keys('울산 '+df.loc[search, 'p_name'])
  # 검색 단추를 누른다
  elem = driver.find_element(By.ID, "searchbox-searchbutton")
  elem.click()
  name = df.loc[search, 'p_name']
  review_address = search_review(driver, df, search, name)
  df.loc[search, 'p_address'] = review_address
  try:
    driver.find_element(By.ID, 'sb_cb50').click()
  except Exception as e:
    driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div/div/div[1]/span/button').click()
    driver.find_element(By.ID, 'sb_cb50').click()

In [ ]:
df.to_csv('kakaoGoogle.csv', encoding='utf-8-sig')